# 앙상블 (Ensemble) : Boosting

- 깊이가 앝은 결정트리를 사용해 이전 트리의 오차를 보정하는 방식
- 순차적으로 경사하강법을 사용해 이전 트리의 오차를 줄여나감
    - 분류 모델에서는 손실함수 Logloss를 사용해 오차를 줄임
    - 회귀 모델에서는 손실함수 MSE를 사용해 오차를 줄임
- Boosting 계열은 일반적으로 결정트리 개수를 늘려도 과적합에 강함
- 대표적인 Boosting 알고리즘(모델) : GradientBoosting, HistGradientBoosting, XGBoost(DMLC), LightGBM(MS), CatBoost

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor

In [6]:
class SimpleGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.initial_prediction = 0     # 초기 예측값
        self.trees = []                 # estimators 모음 배열
    
    def log_odds(self, p):
        return np.log(p / (1 - p))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        y_mean = np.mean(y)
        self.initial_log_adds = self.log_odds(y_mean)
        y_pred_log_odds = np.full(y.shape[0], self.initial_log_adds, dtype=np.float64)
        
        # 개별 모델 생성 및 학습
        for _ in range(self.n_estimators):
            # 현재 상태에서의 예측 확률 계산
            y_pred_proba = self.sigmoid(y_pred_log_odds)
            
            # 잔차(손실) 계산
            residuals = y - y_pred_proba
            
            # 결정 트리 생성 -> 잔차를 라벨로 해서 학습
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.trees.append(tree)
            
            # 예측값 업데이트 (예측값 점진적 개선)
            y_pred_log_odds += self.learning_rate * tree.predict(X)
            
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

    def predict_proba(self, X):
        y_pred_log_odds = np.full(X.shape[0], self.initial_log_adds)
        
        for tree in self.trees:
            y_pred_log_odds += self.learning_rate * tree.predict(X)
        
        return self.sigmoid(y_pred_log_odds)

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

bc_ds = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(bc_ds.data, bc_ds.target, random_state=0)

simple_gb_clf = SimpleGradientBoostingClassifier()
simple_gb_clf.fit(X_train, y_train)

y_pred_train = simple_gb_clf.predict(X_train)
y_pred_test = simple_gb_clf.predict(X_test)

print(f"학습 정확도 : {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도 : {accuracy_score(y_test, y_pred_test)}")

학습 정확도 : 0.9929577464788732
평가 정확도 : 0.958041958041958


In [5]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)

y_pred_train = gb_clf.predict(X_train)
y_pred_test = gb_clf.predict(X_test)
print(f"학습 정확도 : {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도 : {accuracy_score(y_test, y_pred_test)}")

학습 정확도 : 1.0
평가 정확도 : 0.965034965034965


### HistGradientBoosting

- 고성능 GradientBoosting 모델로 대규모 데이터셋 처리에 적합
- Histpgram 기반으로 256개의 구간으로 나누어 처리 병합하는 방식
- 결측치가 있어도 전처리가 필요 없음
- LightGBM 영향을 받아 만들어진 scikit-learn의 모델

In [8]:
from sklearn.ensemble import HistGradientBoostingClassifier

hist_gb_clf = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=3,
    max_bins=255,           # 255개의 구간으로 나누어 처리(1개는 결측치 전용)
    early_stopping=True,    # 학습 반복 중 '일정 횟수' 이상 성능 향상이 없으면 중단
    n_iter_no_change=5      # early_stopping이 True일 때, 성능 향상이 없다고 판단할 '일정 횟수' (기본값 : 10)
)
hist_gb_clf.fit(X_train, y_train)

y_pred_train = hist_gb_clf.predict(X_train)
y_pred_test = hist_gb_clf.predict(X_test)
print(f"학습 정확도 : {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도 : {accuracy_score(y_test, y_pred_test)}")

학습 정확도 : 1.0
평가 정확도 : 0.9790209790209791


In [9]:
from  sklearn.inspection import permutation_importance

result = permutation_importance(
    hist_gb_clf,
    X_train,
    y_train,
    n_repeats=5,            # 실험을 다섯번 하겠다는 뜻
    random_state=0
)

result.importances_mean     # feature별 중요도 평균
result.importances_std      # 중요도의 표준편차
result.importances          # 중요도 (n_features, n_repeats)

{'importances_mean': array([0.        , 0.00187793, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.01032864, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00093897, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01267606, 0.02253521, 0.00187793, 0.        ,
        0.        , 0.        , 0.00187793, 0.        , 0.        ]),
 'importances_std': array([0.        , 0.00175665, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.0028169 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00115   , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00115   , 0.00435381, 0.00175665, 0.        ,
        0.        , 0.        , 0.00093897, 0.        , 0.        ]),
 'importances': array([[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.00234742, 0.        , 0.00234742,

### GradientBoosting [Regression]

In [22]:
from sklearn.datasets import load_diabetes

db_ds = load_diabetes()

db_df = pd.DataFrame(db_ds.data, columns=db_ds.feature_names)
db_df['target'] = db_ds.target

X_train, X_test, y_train, y_test = train_test_split(
    db_ds.data,
    db_ds.target,
    random_state=42
)

In [28]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

hist_gb_reg = HistGradientBoostingRegressor(
    max_iter=200,
    max_depth=1,
    learning_rate=0.1,
    random_state=0,
    l2_regularization=0.0,
    min_samples_leaf=20,
    max_bins=127
)

hist_gb_reg.fit(X_train, y_train)

y_pred_train = hist_gb_reg.predict(X_train)
y_pred_test = hist_gb_reg.predict(X_test)

print(f"학습 MSE : {mean_squared_error(y_train, y_pred_train)}")
print(f"평가 MSE : {mean_squared_error(y_test, y_pred_test)}")
print(f"학습 R2 : {r2_score(y_train, y_pred_train)}")
print(f"평가 R2 : {r2_score(y_test, y_pred_test)}")

학습 MSE : 2348.383005252592
평가 MSE : 2723.2143923397316
학습 R2 : 0.6114923312793334
평가 R2 : 0.5075285428540747
